In [ ]:
# # Module Imports
# import mysql.connector as mariadb
# import sys

# # Connect to MariaDB Platform
# try:
#     mariadb_conn = mariadb.connect(
#         database="conv_project",
#         user="user_ai",
#         password="1234",
#         host="3.104.139.205",
#         port=3306,

#     )
# except mariadb.Error as e:
#     print(f"Error connecting to MariaDB Platform: {e}")
#     sys.exit(1)

# # Get Cursor
# mariadb_cur = mariadb_conn.cursor()

# def insert_smoke_database(CCTV_data):
#     # current_cctv의 데이터를 저장

#     # {
#     #     "latitude": "37.5683",
#     #     "longitude": "126.9778",
#     #     "time": "2021-11-17 13:25:02.066349"
#     # }

#     sqlInsert = "INSERT INTO dashboard_test (LATITUDE, LONGITUDE, TIME) values (%s, %s, %s)"
#     valueInsert = (CCTV_data.get_lat(), CCTV_data.get_long(), CCTV_data.get_time())

#     mariadb_cur.execute(sqlInsert, valueInsert)
#     mariadb_conn.commit()

#     sqlSelect = '''
#         SELECT * FROM dashboard_test ORDER BY id DESC LIMIT 0, 1;
#     '''

#     mariadb_cur.execute(sqlSelect)
#     a = mariadb_cur.fetchall()
#     print(a)

In [ ]:
# sqlSelect = '''
#     SELECT * FROM dashboard_test ORDER BY id DESC LIMIT 0, 1;
# '''

# mariadb_cur.execute(sqlSelect)
# a = mariadb_cur.fetchall()
# print(a)

In [49]:
import tensorflow as tf
tf.config.experimental.list_physical_devices('GPU')
# physical_devices = tf.config.list_physical_devices('GPU') 
# for device in physical_devices:
#     tf.config.experimental.set_memory_growth(device, True)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
# from sklearn.linear_model import LogisticRegression
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.neighbors import KNeighborsClassifier
# from scipy import stats

import json
import numpy as np
import pandas as pd

import io

import os
import matplotlib.image as img
import matplotlib.pyplot as plt
from tqdm import tqdm

from sklearn.model_selection import train_test_split

import easydict
 
args = easydict.EasyDict({
        "batchsize": 1,
        "epoch": 1,
#         "gpu": 0,
        "out": "result",
        "resume": False,
#         "unit": 1000,
        "validationsplit": 0.2,
        "optimizer": "rmsprop",
        "frame": 8,
#         "optimizer": "Adam",
        
})

In [3]:
from numpy import genfromtxt

train_x_data_DF = genfromtxt('./train/train_x.csv', delimiter=',')
train_t_mod_DF = pd.read_csv('./train/train_t.csv', sep=',')

valid_x_data_DF = genfromtxt('./train/valid_x.csv', delimiter=',')
valid_t_mod_DF = pd.read_csv('./train/valid_t.csv', sep=',')

# test_x_data_DF = genfromtxt('/content/drive/MyDrive/6조/train_x_train_t/test_x.csv', delimiter=',')
# test_t_mod_DF = pd.read_csv('/content/drive/MyDrive/6조/train_x_train_t/test_t.csv', sep=',')

# print(train_x_data_DF)
# print(train_x_data_DF.shape)
# print(train_t_mod_DF)

train_x_data = train_x_data_DF
train_t_mod = train_t_mod_DF

valid_x_data = valid_x_data_DF
valid_t_mod = valid_t_mod_DF

# test_x_data = test_x_data_DF
# test_t_mod = test_t_mod_DF

# print(train_x_data)
# print(train_x_data.shape)
# print(type(train_x_data[0][0]))
# print(train_t_mod.shape)

# print(valid_x_data)
# print(valid_x_data.shape)
# print(type(valid_x_data[0][0]))
# print(valid_t_mod.shape)

# [ -1, (한번에 들어갈 입력값 수), (좌표개수) ]
# 13개의 x,y 좌표이므로 26개
train_x = tf.reshape(train_x_data,[-1, args.frame, 26])
train_t = tf.reshape(train_t_mod,[-1,1])

val_x = tf.reshape(valid_x_data,[-1, args.frame, 26])
val_t = tf.reshape(valid_t_mod,[-1,1])

[[ 940.  648.  958. ...  414. 1044.  470.]
 [ 940.  643.  966. ...  414. 1079.  479.]
 [1014.  628. 1005. ...  413. 1061.  469.]
 ...
 [ 904.  963.  929. ...  508. 1013.  683.]
 [ 904.  963.  929. ...  508. 1013.  683.]
 [ 904.  963.  929. ...  508. 1013.  683.]]
(97904, 26)
<class 'numpy.float64'>
(12238, 1)
[[ 920.  844.  915. ...  549.  821.  512.]
 [ 920.  844.  901. ...  587.  790.  643.]
 [ 746.  844.  784. ...  562.  798.  627.]
 ...
 [1113.  597. 1105. ...  324.  990.  394.]
 [1120.  602. 1106. ...  338.  991.  396.]
 [1120.  602. 1127. ...  344. 1026.  408.]]
(41960, 26)
<class 'numpy.float64'>
(5245, 1)


In [6]:
from tensorflow.keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Dropout, Flatten, BatchNormalization, Input, Convolution2D, Activation,TimeDistributed
from tensorflow.keras.layers import Input, multiply, concatenate, Activation, Masking, Reshape
from tensorflow.keras.layers import GlobalAveragePooling1D, Permute, Dropout
from tensorflow.keras.models import Sequential, Model

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

# 정규화하기 minmax

def generate_model():
    ip = Input(shape=(args.frame,26))

    x = Masking()(ip)
    x = LSTM(units = 50, return_sequences = True)(x)
    x = LSTM(units=50)(x)
    x = Dropout(0.3)(x)

  
    y = Reshape((208, 1))(ip)
    y = Conv1D (kernel_size=3, filters=128, strides=3, padding='valid',kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    
    y = Conv1D (kernel_size=3, filters=128, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = MaxPooling1D(pool_size=3, strides=3)(y)

    y = Conv1D (kernel_size=3, filters=128, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = MaxPooling1D(pool_size=3, strides=3)(y)


    y = Conv1D (kernel_size=3, filters=256, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = MaxPooling1D(pool_size=3, strides=3)(y) 

  
    y = Conv1D (kernel_size=3, filters=256, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = MaxPooling1D(pool_size=2, strides=2)(y) 
     
   
    y = Dropout(0.5)(y)
    y = Flatten()(y)
    
    x = concatenate([x, y])

    out = Dense(1, activation='sigmoid')(x)
    

    model = Model(ip, out)
    #model.summary()
    return model
model = generate_model()
# model.summary()

######################################################

early_stopping = EarlyStopping()


model = generate_model()
model.compile(loss='binary_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
history = model.fit(train_x, train_t, epochs=15, batch_size=32,validation_data=(val_x, val_t))
                    # ,callbacks=[early_stopping])
model.save('./model/model_binary_adam_LSTM_FCN.h5')

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 8, 26)]      0                                            
__________________________________________________________________________________________________
reshape (Reshape)               (None, 208, 1)       0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 69, 128)      512         reshape[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 69, 128)      512         conv1d[0][0]                     
_______________________________________________________________________________________

# 목표: Smoke Detection with MQTT connection

- MQTT 통신을 통해 라즈베리파이에서 데이터를 받아오고,
- 카카오브레인에서 제공하는 카카오포즈 api를 활용하여 skeleton 정보를 추출한다.
- 학습시킨 AI 모델을 활용해 흡연 여부를 판별하고,
- 결과가 1에 가까울 시 MQTT통신 및 lambda 함수를 호출해 Database에 결과 저장을 요청한다

In [51]:
# 목표: MQTT 통신을 통해 라즈베리파이에서 데이터를 받아오고,
# 카카오브레인에서 제공하는 카카오포즈 api를 활용하여 skeleton 정보를 추출한다.

# Module Imports

import tensorflow as tf
tf.config.experimental.list_physical_devices('GPU')
# physical_devices = tf.config.list_physical_devices('GPU') 
# for device in physical_devices:
#     tf.config.experimental.set_memory_growth(device, True)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow import keras 
from tensorflow.keras.models import load_model

# from sklearn.linear_model import LogisticRegression
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.neighbors import KNeighborsClassifier
# from scipy import stats

import json
import numpy as np
import pandas as pd

# for communication
import requests, json
from requests import Session
import paho.mqtt.client as mqtt

# for data transforamtion
import os
import io
import cv2
import base64
from tempfile import TemporaryFile
from PIL import Image
import matplotlib.image as img
import matplotlib.pyplot as plt
from tqdm import tqdm

from sklearn.model_selection import train_test_split

import easydict
 
args = easydict.EasyDict({
        "batchsize": 1,
        "epoch": 1,
#         "gpu": 0,
        "out": "result",
        "resume": False,
#         "unit": 1000,
        "validationsplit": 0.2,
        "optimizer": "rmsprop",
        "frame": 8,
#         "optimizer": "Adam",
        
})

- 테스트를 위해 json 값을 출력하는 함수

In [10]:
# print .json file for testing
def print_json(obj):
    """Print the object as json"""
    print(json.dumps(obj, sort_keys=True, indent=2, separators=(',', ': ')))

## CCTV로부터 도착한 위도/경도/이미지 값을 저장하기 위한 객체 선언

In [18]:
# import io
# import cv2
# import base64
# from tempfile import TemporaryFile
# from PIL import Image

class CctvData:
    def __init__(self, latitude, longitude, time, image):
        self._latitude = latitude
        self._longitude = longitude
        self._time = time
        self._image = image
        self._image_fp = self.img_byte_to_png_file(image)
    
    def __str__(self):
        data_string = "CCTV at : "+ self._time + "\n" \
                    + "latitude : "+ self._latitude + "\n" \
                    + "longitude : "+ self._longitude + "\n"
#                     + "time : "+ self._image + "\n" \
        return data_string
    

    def get_img(self):
        return self._image
    
    def get_png(self):
        return self._image_fp
    
    def close_fp(self):
        self._image_fp.close()
    
    def get_lat(self):
        return self._latitude
    
    def get_long(self):
        return self._longitude
    
    def get_time(self):
        return self._time
    
    
#     def set_img(self, image):
#         self._image = image
#         return 0


    def img_byte_to_png_file(self, image):
        imgdata = base64.b64decode(str(self._image))
#         print( type(imgdata) ) # <class 'bytes'>

        image = Image.open(io.BytesIO(imgdata))
#         image = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2RGB) # not required

        img_fp = TemporaryFile()
        image.save(img_fp, "PNG")
        img_fp.seek(0)

        return img_fp

- Kakao Pose API를 요청하는 함수

In [20]:
# import requests, json
# from requests import Session

with open("./Kakao-API-Key.txt") as REST_API_KEY:
    APP_KEY = REST_API_KEY.readline()
    REST_API_KEY.close()

# APP_KEY = REST_API_KEY
session = Session()
session.headers.update({'Authorization': 'KakaoAK ' + APP_KEY})


def request_kakao_api(path_data):
    response = session.post('https://cv-api.kakaobrain.com/pose', files={'file': path_data})
    response.raise_for_status()
    return response.json()


- Database에 저장을 요청하는 함수

In [19]:
# import requests, json
# from requests import Session

def request_smoke_database(CCTV_data):
    # current_cctv의 데이터를 send format에 맞게 수정 후 저장 요청
    
    # JSON Send Format:
    # {
    #     "latitude": "37.5683",
    #     "longitude": "126.9778",
    #     "time": "2021-11-17 13:25:02.066349"
    # }

    data={"latitude":CCTV_data.get_lat(), "longitude":CCTV_data.get_long(),"time":CCTV_data.get_time()}
    
    URL= "https://6eocezhdhb.execute-api.ap-southeast-2.amazonaws.com/v2/dbConnectFunction"
    ret_val= requests.post(URL, data=json.dumps(data))

## Keypoint의 순서를 바꿔주는 함수
- 학습에 사용한 AI Hub의 keypoint 데이터 형식에 맞게 Kakao API에서 받아온 결과를변환한다

### KAKAO API RESULT :  COCO(Common Objects in Context) dataformat : 17개
```
1	nose 9
    2	left_eye (not used) 99
    3	right_eye (not used) 99
    4	left_ear (not used) 99
    5	right_ear (not used) 99 
6	left_shoulder 13
7	right_shoulder 12
8	left_elbow 14
9	right_elbow 11
10	left_wrist 15
11	right_wrist 10
12	left_hip 3 
13	right_hip 2
14	left_knee 4
15	right_knee 1
16	left_ankle 5
17	right_ankle 0
```


### for AI Hub dataformat : 16개
```
0 right_ankle 17
1 right_knee 15
2 right_hip 13
3 left_hip 12
4 left_knee 14
5 left_ankle 16
    6: hip(not used)
    7: chest(not used)
    8: neck
9 head (nose 1)
10 right_wrist 11
11 right_elbow 9
12 right_shoulder 7
13 left_shoulder 6
14 left_elbow 8
15 left_wrist 10
```

In [21]:
# Converting Joint infomation

# to AI Hub dataformat
def convert_annotation_to_aihub(keypoint):
    COCO_TO_AI = (
        (0,9),  (1,99),
        (2,99), (3,99),
        (4,99), (5,13),
        (6,12), (7,14),
        (8,11), (9,15),
        (10,10),(11,3),
        (12,2), (13,4),
        (14,1), (15,5),
        (16,0)
    )
    KakaoCOCO = 0
    AIHUB = 1
    
    new_keypoint = np.zeros(shape=(16,3))
#     print(new_keypoint)
    
    for cnt in range(0, 17):
        AI = COCO_TO_AI[cnt][AIHUB]
        COCO = COCO_TO_AI[cnt][KakaoCOCO]
#         print(str(cnt) + ' ')
#         print(str(COCO) + ' ' + str(AI))
        
        if (AI==99):
            continue
        new_keypoint[AI][0] = keypoint[COCO][0]
        new_keypoint[AI][1] = keypoint[COCO][1]
        new_keypoint[AI][2] = keypoint[COCO][2]*100
    
#     keypoint_int = np.round(new_keypoint, 0)
#     keypoint_int = np.asarray(keypoint_int, dtype = int)
#     return keypoint_int
    return new_keypoint
    
    
        
# to KAKAO API RESULT :  COCO(Common Objects in Context) dataformat
def convert_annotation_to_coco(keypoint):
    AI_TO_COCO = (
        (0,17-1), (1,15-1),
        (2,13-1), (3,12-1),
        (4,14-1), (5,16-1),
        (6,99),   (7,99),
        (8,99),   (9,1-1),
        (10,11-1),(11,9-1),
        (12,7-1), (13,6-1),
        (14,8-1), (15,10-1)
    )
    AIHUB = 0
    KakaoCOCO = 1
#     print(len(keypoint))
    
    new_keypoint = np.zeros(shape=(16,3))
#     print(new_keypoint)
    
    for cnt in range(0, 16):
        AI = AI_TO_COCO[cnt][AIHUB]
        COCO = AI_TO_COCO[cnt][KakaoCOCO]
        
        if (COCO==99):
            continue

        new_keypoint[COCO][0] = keypoint[AI][0]
        new_keypoint[COCO][1] = keypoint[AI][1]
        new_keypoint[COCO][2] = keypoint[AI][2]
    
#     keypoint_int = np.round(new_keypoint, 0)
#     keypoint_int = np.asarray(keypoint_int, dtype = int)
#     return keypoint_int
    return new_keypoint


## 사용하지 않는 annotation 값 제거
- 대상: 6, 7, 8
    - 6: hip(not used)
    - 7: chest(not used)
    - 8: neck

In [22]:
def remove_unused_annotation(keypoint):
    # 학습데이터는 모두 2D데이터
    # 따라서, 사용하지 않는 값 (_, _, 2)을 제거
    keypoint = keypoint.reshape(-1, 3)
    result = np.delete(keypoint, 2, axis=1)
    
    # 실제 예측에서 사용할 수 없는 Annotation 제거
    result = np.delete(result, [6,7,8], axis=0)
    result = result.reshape(1,-1)
    
    return result

# on_message(): 라즈베리파이로부터 데이터를 받아와 predict 후 결과를 송출
- 수신한 데이터를 한 곳에 모은 뒤 8개가 모이면 prediction을 진행
- 8개 미만으로 도착할 경우 (임시로) 데이터를 버림
- predict 결과가 0.7 미만일 시 False, 이상일 시 True로 판단
- 판단 결과에 따라 True일 경우 위경도 값과 시간을 lambda로 전송

In [ ]:
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))
    if rc == 0:
        client.subscribe("smoke/cctv")   # 연결 성공시 토픽 구독 신청
        # client.subscribe("smoke/signal") # iot 측에서 구독할 토픽
    else:
        print('연결실패: ',rc)

keypoint_data_list = np.zeros(shape=(args.frame,26))
count = 0
zero_cnt = 0
print('count: {} / zero_cnt: {}'.format(count, zero_cnt))

In [47]:
def on_message(client, userdata, msg):
    json_recieved = json.loads(msg.payload)
    global count
    global keypoint_data_list
    global zero_cnt
    
#     JSON Recieve Format:
#     {
#         "latitude": "37.5683",
#         "longitude": "126.9778",
#         "time": "2021-11-17 13:25:02.066349",
#         "image": "/9j/4AAQSkZ"
#     }
    latitude = json_recieved.get("latitude")
    longitude = json_recieved.get("longitude")
    time_cctv = json_recieved.get("time")
    img_cctv = json_recieved.get("image")

    # 데이터 관리를 위한 CCTV객체 생성
    # CCTV객체는 byte배열을 이미지 형태로 변환,
    # TemporaryFile()을 이용해 임시 파일 포인터로 png를 가지고 있다
    current_cctv = CctvData(latitude, longitude, time_cctv, img_cctv)
#     print(current_cctv)    
    
    # KakapPose api 요청
    result = request_kakao_api(current_cctv.get_png())
    current_cctv.close_fp() # 더이상 사용하지 않으므로 file close()
    
#     print_json(result)
#     print(len(result))
    
    answer = 0
    if len(result)>0:
        # 사람이 감지되면 수행
        
        # [x, y, w, h]
#         body_box = np.array(result[0]['bbox'])

        # [x_1, y_1, score_1, x_2, y_2, score_2, ..., x_17, y_17, score_17]
        keypoint = np.array(result[0]['keypoints'])
        # 신뢰도 reliability
        score = result[0]['score']

        keypoint_mod = keypoint.reshape(-1, 3)
        print(len(keypoint_mod))

        # Kakao API를 통해 받아온 annotation을 순서에 맞게 조정
        keypoint_data = convert_annotation_to_aihub(keypoint_mod)

        # 인공지능에게 body_box, score, keypoint_data를 던져준다
####################################################

        keypoint_data_list[count] = remove_unused_annotation(keypoint_data)
        count += 1
        print(count)
               
        if (count == args.frame) :
#             model = load_model('./model/model_binary_adam_LSTM_FCN.h5')
            keypoint_data_list = keypoint_data_list*2
            answer = model.predict(keypoint_data_list.reshape(-1, 8, 26))
            count = 0

        # AI의 판단 결과를 IoT에게 발송
        # IoT는 결과값이 True로 바뀌게 될 경우 알람 방송
        if (answer >= 0.7):
            print("yes: {}".format(answer))
            client.publish('smoke/signal',1)
            request_smoke_database(current_cctv)
        else:
#             answer = 0
            print("Noooo: {}".format(answer))

####################################################

    zero_cnt += 1
    if zero_cnt==8 :
        count=0
        zero_cnt=0
        # 8개마다 변수 초기화
        if (answer < 0.7) :
            print("Signal sent : 0")
            client.publish('smoke/signal',0)

#         else : 
#             # 사람이 없을 때 pass
#             print("none")
# #             request_smoke_database(current_cctv)
# #             mc.publish('smoke/signal',f"{answer}")
#             client.publish('smoke/signal',0)

#     else:
#         answer = 0
        
    # if (answer >= 0.8):
    #     print("yes")
    #     client.publish('smoke/signal',1)
    #     request_smoke_database(current_cctv)

    # else : 
    # answer == 0
    # 사람이 없을 때 pass

count: 0 / zero_cnt: 0


# Main
- MQTT Broker에 연결 후 이벤트 발생 시 on_message()함수를 수행

In [54]:
if __name__ == "__main__":
    # 1. MQTT 클라이언트 객체 인스턴스화
    client = mqtt.Client()

    # 2. 관련 이벤트에 대한 콜백 함수 등록
    client.on_connect = on_connect
    client.on_message = on_message
    
#     keypoint_data_list = np.array([])
    
    try:
        # 3. 브로커 연결
        client.connect("18.180.148.242", port=5555)

        # 4. 메시지 루프 - 이벤트 발생시 해당 콜백 함수 호출
        client.loop_forever()

    except Exception as err:
        print("에러 : %s"%err)

Connected with result code 0
CCTV at : 2021-12-08 17:09:00.020351
latitude : 37.509655
longitude : 127.05589

17
6
Noooo: 0
CCTV at : 2021-12-08 17:09:00.655668
latitude : 37.509655
longitude : 127.05589

17
7
Noooo: 0
CCTV at : 2021-12-08 17:09:01.118846
latitude : 37.509655
longitude : 127.05589

17
8
Noooo: [[0.08363181]]
Signal sent : 0
CCTV at : 2021-12-08 17:09:01.561671
latitude : 37.509655
longitude : 127.05589

17
1
Noooo: 0
CCTV at : 2021-12-08 17:09:02.006402
latitude : 37.509655
longitude : 127.05589

17
2
Noooo: 0
CCTV at : 2021-12-08 17:09:02.456177
latitude : 37.509655
longitude : 127.05589

17
3
Noooo: 0
CCTV at : 2021-12-08 17:09:02.898060
latitude : 37.509655
longitude : 127.05589

17
4
Noooo: 0
CCTV at : 2021-12-08 17:09:03.339485
latitude : 37.509655
longitude : 127.05589

17
5
Noooo: 0
CCTV at : 2021-12-08 17:09:06.299065
latitude : 37.509605
longitude : 127.055905

17
6
Noooo: 0
CCTV at : 2021-12-08 17:09:06.749349
latitude : 37.509605
longitude : 127.055905

17
7

17
8
Noooo: [[6.446587e-05]]
Signal sent : 0
CCTV at : 2021-12-08 17:10:25.753709
latitude : 37.509493
longitude : 127.056163

17
1
Noooo: 0
CCTV at : 2021-12-08 17:10:26.196336
latitude : 37.509493
longitude : 127.056163

17
2
Noooo: 0
CCTV at : 2021-12-08 17:10:26.640961
latitude : 37.509493
longitude : 127.056163

17
3
Noooo: 0
CCTV at : 2021-12-08 17:10:27.085018
latitude : 37.509493
longitude : 127.056163

17
4
Noooo: 0
CCTV at : 2021-12-08 17:10:27.528537
latitude : 37.509493
longitude : 127.056163

17
5
Noooo: 0
CCTV at : 2021-12-08 17:10:32.173106
latitude : 37.50953
longitude : 127.056158

17
6
Noooo: 0
CCTV at : 2021-12-08 17:10:32.646184
latitude : 37.50953
longitude : 127.056158

17
7
Noooo: 0
CCTV at : 2021-12-08 17:10:33.088991
latitude : 37.50953
longitude : 127.056158

17
8
Noooo: [[0.00062853]]
Signal sent : 0
CCTV at : 2021-12-08 17:10:33.533200
latitude : 37.50953
longitude : 127.056158

17
1
Noooo: 0
CCTV at : 2021-12-08 17:10:33.977322
latitude : 37.50953
longitude

17
3
Noooo: 0
CCTV at : 2021-12-08 17:11:50.730823
latitude : 37.50959
longitude : 127.056052

17
4
Noooo: 0
CCTV at : 2021-12-08 17:11:51.179039
latitude : 37.50959
longitude : 127.056052

17
5
Noooo: 0
CCTV at : 2021-12-08 17:11:56.242222
latitude : 37.509625
longitude : 127.056088

17
6
Noooo: 0
CCTV at : 2021-12-08 17:11:56.700331
latitude : 37.509625
longitude : 127.056088

17
7
Noooo: 0
CCTV at : 2021-12-08 17:11:57.143479
latitude : 37.509625
longitude : 127.056088

17
8
Noooo: [[0.25431168]]
Signal sent : 0
CCTV at : 2021-12-08 17:11:57.588304
latitude : 37.509625
longitude : 127.056088

17
1
Noooo: 0
CCTV at : 2021-12-08 17:11:58.032698
latitude : 37.509625
longitude : 127.056088

17
2
Noooo: 0
CCTV at : 2021-12-08 17:11:58.480067
latitude : 37.509625
longitude : 127.056088

17
3
Noooo: 0
CCTV at : 2021-12-08 17:11:58.927950
latitude : 37.509625
longitude : 127.056088

17
4
Noooo: 0
CCTV at : 2021-12-08 17:11:59.379996
latitude : 37.509625
longitude : 127.056088

17
5
Noooo: 0

KeyboardInterrupt: 

In [21]:
keypoint_data_list = np.zeros(shape=(args.frame,26))
count = 0

for i in range(0,args.frame):
    data = np.array(range(0,26))
#     print(data.shape)

    keypoint_data_list[count] = data.reshape(1,-1)
    count += 1
#     keypoint_data_list = np.insert(keypoint_data_list[1], data, axis=0)

print(keypoint_data_list.shape)

(8, 26)


In [32]:
from numpy import genfromtxt

test_X = genfromtxt('./train/test_x.csv', delimiter=',')
test_x_data = test_X.reshape(-1, args.frame, 26)

valid_t_mod_DF = pd.read_csv('./train/test_t.csv', sep=',')

print(test_x_data[0].shape)
# X = tf.reshape(test_x_data[0],[-1, args.frame, 26])

# model = load_model('./model/model_0.h5')
model.summary()
for layer in model.layers:
    print(layer.input_shape)

(8, 26)
Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 8, 26)]      0                                            
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 208, 1)       0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 69, 128)      512         reshape_1[0][0]                  
__________________________________________________________________________________________________
batch_normalization_5 (BatchNor (None, 69, 128)      512         conv1d_5[0][0]                   
_______________________________________________________________________________

In [39]:
INDEX = 100

answer = model.predict(test_x_data[INDEX].reshape(-1, 8, 26))
print(answer)
print(valid_t_mod_DF.loc[INDEX])

[[0.9999918]]
Answer    1
Name: 100, dtype: int64


In [41]:
test1 = np.array([[297.5,32.5,0.879,303.5,26.5,0.8567,290.5,21.5,0.8863,259.5,19.5,0.004,259.5,19.5,0.8719,302.5,100.5,0.7,211.5,75.5,0.7201,306.5,171.5,0.5957,212.5,141.5,0.727,322.5,216.5,0.7158,267.5,91.5,0.7563,257.5,254.5,0.5293,204.5,244.5,0.5735,234.5,364.5,0.4814,187.5,350.5,0.609,215.5,454.5,0.3623,175.5,451.5,0.6835],
[305.5,32.5,0.8624,311.5,26.5,0.8291,299.5,21.5,0.8895,311.5,25.5,0.0041,267.5,18.5,0.8795,307.5,101.5,0.699,220.5,76.5,0.7013,309.5,176.5,0.6466,221.5,146.5,0.7034,305.5,244.5,0.7124,277.5,97.5,0.7308,268.5,253.5,0.5297,217.5,244.5,0.5589,243.5,363.5,0.4857,198.5,354.5,0.5895,227.5,451.5,0.1931,176.5,446.5,0.6656],
[321.5,29.5,0.8785,326.5,23.5,0.7708,316.5,17.5,0.8941,282.5,13.5,0.004,282.5,13.5,0.8592,314.5,99.5,0.6636,233.5,73.5,0.6667,315.5,180.5,0.6082,227.5,148.5,0.7255,323.5,252.5,0.6971,291.5,104.5,0.7141,274.5,255.5,0.4782,230.5,248.5,0.4561,247.5,369.5,0.3832,221.5,368.5,0.3903,243.5,459.5,0.1115,207.5,474.5,0.196],
[340.5,18.5,0.8304,343.5,9.5,0.0481,337.5,6.5,0.8293,304.5,5.5,0.0048,303.5,2.5,0.5054,309.5,84.5,0.6133,244.5,65.5,0.6193,308.5,169.5,0.4033,245.5,147.5,0.6957,321.5,243.5,0.6402,330.5,124.5,0.6705,275.5,251.5,0.3694,235.5,246.5,0.4214,274.5,378.5,0.1873,227.5,376.5,0.3682,227.5,467.5,0.0644,206.5,474.5,0.1958],
[321.5,29.5,0.8784,326.5,23.5,0.7696,316.5,17.5,0.8941,282.5,13.5,0.004,282.5,13.5,0.8592,314.5,99.5,0.6633,233.5,73.5,0.6664,315.5,180.5,0.608,227.5,148.5,0.7254,323.5,252.5,0.6968,291.5,104.5,0.7142,274.5,255.5,0.4777,230.5,248.5,0.4563,247.5,369.5,0.3829,221.5,368.5,0.3902,243.5,459.5,0.1114,207.5,474.5,0.196],
[305.5,32.5,0.8622,311.5,26.5,0.8289,299.5,21.5,0.8894,311.5,25.5,0.0041,267.5,18.5,0.8791,307.5,101.5,0.6991,220.5,76.5,0.7012,309.5,176.5,0.646,221.5,146.5,0.7033,305.5,244.5,0.7119,277.5,97.5,0.7307,268.5,253.5,0.5296,217.5,244.5,0.5586,243.5,363.5,0.4853,198.5,354.5,0.5896,227.5,451.5,0.1932,176.5,446.5,0.6658],
[297.5,32.5,0.8789,303.5,26.5,0.8575,290.5,21.5,0.8864,259.5,19.5,0.004,259.5,19.5,0.8718,302.5,100.5,0.7003,211.5,75.5,0.7204,306.5,171.5,0.5956,212.5,141.5,0.7276,322.5,216.5,0.7158,267.5,91.5,0.7561,257.5,254.5,0.5296,204.5,244.5,0.5733,234.5,364.5,0.4814,187.5,350.5,0.609,215.5,454.5,0.3625,175.5,451.5,0.6832],
[305.5,32.5,0.8622,311.5,26.5,0.8281,299.5,21.5,0.8895,311.5,25.5,0.0041,267.5,18.5,0.8792,307.5,101.5,0.6987,220.5,76.5,0.7012,309.5,176.5,0.6459,221.5,146.5,0.703,305.5,244.5,0.7123,277.5,97.5,0.7304,268.5,253.5,0.53,217.5,244.5,0.5589,243.5,363.5,0.4863,198.5,354.5,0.5896,227.5,451.5,0.1927,176.5,446.5,0.6653]])

test2 = np.array([[297.5,32.5,0.8792,303.5,26.5,0.8572,290.5,21.5,0.8866,259.5,19.5,0.004,259.5,19.5,0.8719,302.5,100.5,0.7,211.5,75.5,0.7202,306.5,171.5,0.5958,212.5,141.5,0.7271,322.5,216.5,0.7154,267.5,91.5,0.7563,257.5,254.5,0.5293,204.5,244.5,0.5734,234.5,364.5,0.4814,187.5,350.5,0.6086,215.5,454.5,0.3628,175.5,451.5,0.6836],
[305.5,32.5,0.8622,311.5,26.5,0.8289,299.5,21.5,0.8895,311.5,25.5,0.0041,267.5,18.5,0.8792,307.5,101.5,0.6988,220.5,76.5,0.7012,309.5,176.5,0.6464,221.5,146.5,0.704,305.5,244.5,0.7124,277.5,97.5,0.7306,268.5,253.5,0.5298,217.5,244.5,0.5587,243.5,363.5,0.4855,198.5,354.5,0.5892,227.5,451.5,0.1933,176.5,446.5,0.6654],
[321.5,29.5,0.8784,326.5,23.5,0.7696,316.5,17.5,0.8941,282.5,13.5,0.004,282.5,13.5,0.8592,314.5,99.5,0.6633,233.5,73.5,0.6664,315.5,180.5,0.608,227.5,148.5,0.7254,323.5,252.5,0.6968,291.5,104.5,0.7142,274.5,255.5,0.4777,230.5,248.5,0.4563,247.5,369.5,0.3829,221.5,368.5,0.3902,243.5,459.5,0.1114,207.5,474.5,0.196],
[340.5,18.5,0.8309,343.5,9.5,0.048,337.5,6.5,0.8289,304.5,5.5,0.0048,303.5,2.5,0.5052,309.5,84.5,0.6131,244.5,65.5,0.6193,308.5,168.5,0.4031,245.5,147.5,0.6956,321.5,243.5,0.6403,330.5,124.5,0.6703,275.5,251.5,0.3695,235.5,246.5,0.4216,274.5,378.5,0.1871,227.5,376.5,0.3679,227.5,467.5,0.0645,206.5,474.5,0.1959],
[396.5,56.5,0.8013,400.5,46.5,0.0171,397.5,42.5,0.833,364.5,25.5,0.004,363.5,25.5,0.8591,333.5,102.5,0.5795,291.5,76.5,0.582,326.5,178.5,0.3051,282.5,176.5,0.6856,361.5,256.5,0.5546,368.5,206.5,0.6636,248.5,250.5,0.3433,221.5,243.5,0.398,258.5,371.5,0.1998,222.5,370.5,0.2362,194.5,447.5,0.0789,193.5,452.5,0.1231],
[418.5,87.5,0.8225,424.5,77.5,0.0069,421.5,73.5,0.8612,389.5,54.5,0.004,389.5,54.5,0.8554,348.5,117.5,0.5389,314.5,104.5,0.5337,352.5,195.5,0.5734,327.5,195.5,0.692,392.5,265.5,0.3442,389.5,262.5,0.6261,234.5,246.5,0.3094,218.5,242.5,0.4649,233.5,361.5,0.4482,216.5,362.5,0.498,139.5,437.5,0.511,210.5,469.5,0.2669],
[407.5,94.5,0.8299,411.5,84.5,0.0441,409.5,80.5,0.8822,375.5,62.5,0.0041,374.5,62.5,0.879,337.5,120.5,0.4974,300.5,114.5,0.4901,351.5,194.5,0.5832,326.5,201.5,0.6482,380.5,271.5,0.4442,387.5,274.5,0.6585,214.5,245.5,0.4022,205.5,243.5,0.484,199.5,363.5,0.4076,209.5,363.5,0.4409,142.5,461.5,0.3971,205.5,467.5,0.2153],
[386.5,111.5,0.8292,392.5,101.5,0.024,388.5,97.5,0.878,354.5,79.5,0.0041,354.5,79.5,0.8786,315.5,131.5,0.4119,282.5,125.5,0.553,341.5,202.5,0.5682,320.5,205.5,0.5933,379.5,270.5,0.3478,390.5,275.5,0.6404,173.5,238.5,0.3617,168.5,236.5,0.4315,165.5,361.5,0.2979,193.5,368.5,0.3429,134.5,471.5,0.1989,178.5,467.5,0.0949]])


new_test1 = np.zeros(shape=(8,26))
for i in range(0,8):
    temp = convert_annotation_to_aihub(test1[i].reshape(-1, 3))
    new_test1[i] =  remove_unused_annotation(temp.reshape(1,-1))

print(test1.shape)
print(new_test1.shape)
# visualize_data(new_test1)


new_test2 = np.zeros(shape=(8,26))
for i in range(0,8):
    temp = convert_annotation_to_aihub(test2[i].reshape(-1, 3))
    new_test2[i] =  remove_unused_annotation(temp.reshape(1,-1))

new_test3 = new_test1 * 2
new_test4 = new_test2 * 2

print(test2.shape)
print(new_test2.shape)
# test_mul = new_test1*2

# print(test_mul.min())
# print(test_mul.max())

answer = model.predict(new_test3.reshape(-1, 8, 26))
print("Answer : {}".format(answer))


answer = model.predict(new_test4.reshape(-1, 8, 26))
print("Answer : {}".format(answer))

# print(new_test3)
# print(new_test4)

(8, 51)
(8, 26)
(8, 51)
(8, 26)
Answer : [[0.9435283]]
Answer : [[0.998965]]


In [43]:
with open('./test_data/asdf.json', 'r', newline='') as jsonfile:
    json_recieved = json.load(jsonfile)

print(len(json_recieved))
print(len(json_recieved[0]['keypoints']))

test_keyp = np.zeros(shape=(39,51))
for i in range(0, len(json_recieved)):
    test_keyp[i] = json_recieved[i]['keypoints']

print(test_keyp.shape)

new_test5 = np.zeros(shape=(39,26))
for i in range(0,len(test_keyp)):
#     print(test_keyp[i])
    temp = convert_annotation_to_aihub(test_keyp[i].reshape(-1, 3))
#     print(temp)
    new_test5[i] =  remove_unused_annotation(temp.reshape(1,-1))
#     print(new_test1[i])

INDEX = 0

print(new_test5.shape)
# print(new_test5.min())
# print(new_test5.max())
# visualize_data(new_test1[0:1])
new_test5 = new_test5[INDEX:INDEX+8]
print(new_test5.shape)

39
51
(39, 51)
(39, 26)
(8, 26)


In [44]:
answer = model.predict(new_test5.reshape(-1, 8, 26))
print("Answer : {}".format(answer))

Answer : [[0.00063153]]
